In [15]:
import Axios from 'axios';

const loadPage = async (title: string) => {
  const SEARCH_URL = "https://www.amazon.com/s";
  const result = await Axios.get(SEARCH_URL, {
    params: {
      i: 'digital-text',
      k: title,
    },
    validateStatus: () => true,
    headers: {'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-gpc': '1',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9'}
  });
  return result.data;
};

In [16]:
import * as cheerio from 'cheerio';

const parseResults = (data: string) => {
  const $ = cheerio.load(data);

  const results = [];

  $('div[data-component-type=s-search-result]').each((i, elem) => {
    const e = $(elem);
    const asin = e.attr('data-asin');
    const imageRecord = e.find('img[class=s-image]').first();
    const title = imageRecord.attr('alt');
    const imageUrl = imageRecord.attr('src');
    results.push({ asin, title, imageUrl });
  });

  return results;
}

In [17]:
loadPage('Jurassic Park').then(parseResults).then(console.info);

Promise { <pending> }
[
  {
    asin: 'B007UH4D3G',
    title: 'Jurassic Park: A Novel',
    imageUrl: 'https://m.media-amazon.com/images/I/81b2mR9dMWL._AC_UY218_.jpg'
  },
  {
    asin: 'B000FC1J76',
    title: 'The Lost World: A Novel (Jurassic Park Book 2)',
    imageUrl: 'https://m.media-amazon.com/images/I/716pwpM4AFL._AC_UY218_.jpg'
  },
  {
    asin: 'B07692WFHD',
    title: 'Jurassic Park: The Deluxe Novelization (Jurassic Park)',
    imageUrl: 'https://m.media-amazon.com/images/I/81zBk5azirL._AC_UY218_.jpg'
  },
  {
    asin: 'B08D7WH4VJ',
    title: 'Jurassic Park: Dead Islands',
    imageUrl: 'https://m.media-amazon.com/images/I/91xbnvaKBML._AC_UY218_.jpg'
  },
  {
    asin: 'B0776JX3GS',
    title: 'Jurassic Park Little Golden Book (Jurassic Park)',
    imageUrl: 'https://m.media-amazon.com/images/I/91d3LsQpWZL._AC_UY218_.jpg'
  },
  {
    asin: 'B086RPSBBK',
    title: 'Jurassic Park: Trivia Quiz Book',
    imageUrl: 'https://m.media-amazon.com/images/I/91Zz2X9FoxL._AC_UY2